In [1]:
import pandas as pd
import numpy as np
import warnings
from math import sqrt
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
 

In [2]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='gabe-lmps'
data_key = 'mean_hourly_solar2.csv'
data_location = 's3://{0}/{1}'.format(bucket, data_key)
mea_hr_s = pd.read_csv(data_location)

In [3]:
mea_hr_s=mea_hr_s.drop(['pnode_id', 'row_is_current', 'version_nbr', 'is_renewable'], axis=1)

In [4]:
mea_hr_s=mea_hr_s.set_index('datetime')
mea_hr_s.head()

,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,solar_generation_mw,mw,fuel_percentage_of_total
datetime,,,,,,,
2020-07-01 00:00:00,15.65,15.672963,0.00,0.019676,-1.153,21.9,0.0
2020-07-01 01:00:00,15.24,15.276157,0.00,0.034815,-1.157,21.9,0.0
2020-07-01 02:00:00,14.43,14.498889,0.03,0.040324,-1.190,21.9,0.0
2020-07-01 03:00:00,13.66,13.704028,0.00,0.041343,-1.172,21.9,0.0
2020-07-01 04:00:00,13.44,13.482500,0.00,0.041343,-1.180,21.9,0.0


In [ ]:

#code adapdted from https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/
#authored by Jason Brownlee
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 


# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(mea_hr_s["total_lmp_rt"].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=13.431
ARIMA(0, 0, 1) RMSE=12.172
ARIMA(0, 0, 2) RMSE=11.808
ARIMA(0, 1, 0) RMSE=13.155
ARIMA(0, 1, 1) RMSE=11.350
ARIMA(0, 1, 2) RMSE=11.371
ARIMA(0, 2, 0) RMSE=22.317
ARIMA(0, 2, 1) RMSE=13.158
ARIMA(0, 2, 2) RMSE=11.355
ARIMA(1, 0, 0) RMSE=11.483
ARIMA(1, 0, 1) RMSE=11.157
ARIMA(1, 0, 2) RMSE=11.162
ARIMA(1, 1, 0) RMSE=11.933
ARIMA(1, 1, 1) RMSE=11.409
ARIMA(1, 1, 2) RMSE=11.119
ARIMA(1, 2, 0) RMSE=17.230
ARIMA(1, 2, 1) RMSE=11.936
ARIMA(1, 2, 2) RMSE=11.418
ARIMA(2, 0, 0) RMSE=11.204
ARIMA(2, 0, 1) RMSE=11.162
ARIMA(2, 0, 2) RMSE=11.129
ARIMA(2, 1, 0) RMSE=11.546
ARIMA(2, 1, 1) RMSE=11.160
ARIMA(2, 1, 2) RMSE=11.260
ARIMA(2, 2, 0) RMSE=14.911
ARIMA(2, 2, 1) RMSE=11.550
ARIMA(4, 0, 0) RMSE=11.162
ARIMA(4, 0, 1) RMSE=11.162
ARIMA(4, 0, 2) RMSE=11.058
ARIMA(4, 1, 0) RMSE=11.469
ARIMA(4, 1, 2) RMSE=11.131
ARIMA(4, 2, 0) RMSE=13.267
ARIMA(4, 2, 1) RMSE=11.473
ARIMA(6, 0, 0) RMSE=11.185
ARIMA(6, 0, 1) RMSE=11.416
ARIMA(6, 1, 0) RMSE=11.448
ARIMA(6, 1, 1) RMSE=11.129
A

In [ ]:

#code adapdted from https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/
#authored by Jason Brownlee
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 


# evaluate parameters
p_values = [8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(mea_hr_s["total_lmp_rt"].values, p_values, d_values, q_values)

ARIMA(8, 0, 0) RMSE=11.218
ARIMA(8, 0, 1) RMSE=11.422
ARIMA(8, 1, 0) RMSE=11.431
ARIMA(8, 1, 1) RMSE=11.127
ARIMA(8, 1, 2) RMSE=11.141
ARIMA(8, 2, 0) RMSE=12.518
ARIMA(8, 2, 1) RMSE=11.434
ARIMA(8, 2, 2) RMSE=11.470
ARIMA(10, 0, 0) RMSE=11.307
ARIMA(10, 0, 1) RMSE=11.474
ARIMA(10, 1, 0) RMSE=11.381
ARIMA(10, 1, 1) RMSE=11.177
ARIMA(10, 1, 2) RMSE=11.145
ARIMA(10, 2, 0) RMSE=12.371


In [5]:
#code adapdted from https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/
#authored by Jason Brownlee
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 


# evaluate parameters
p_values = [10]
d_values = [2]
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(mea_hr_s["total_lmp_rt"].values, p_values, d_values, q_values)

ARIMA(10, 2, 0) RMSE=12.371
ARIMA(10, 2, 1) RMSE=11.385
ARIMA(10, 2, 2) RMSE=11.413
Best ARIMA(10, 2, 1) RMSE=11.385
